In [1]:
import sys
from openai import OpenAI

import os
from typing import List, Union
from pydantic import BaseModel
from git import Repo, GitCommandError

sys.path.append("C:\\Users\\SHIRAM\\PycharmProjects\\LLMStuff\\code")
from utils import str_to_path, get_message_obj
from utils import init_chat_conversation, convert_history_to_convo

from typing import List
from blocks import (
    get_repo_path_textbox,
    get_max_tokens_slider,
    get_model_names_dropdown,
    get_prompt_options,
)

ROOT_PATH = "C:\\Users\\SHIRAM"
root_path = str_to_path(path=ROOT_PATH)
rep_path = str_to_path(path=f"{ROOT_PATH}\\Documents\\markinghelperAI")


def get_chosen_files(repo_path: str) -> List[str]:
    return gr.FileExplorer(
        label="Repository Path",
        glob="**/*.py",
        file_count="multiple",
        root=repo_path,
        show_label=False,
    )


def select_chat_window():
    return gr.Tabs(selected=1)

c:\Users\shrirams\Documents\LLMTools\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
repo = Repo(root_path / "PycharmProjects" / "LLMStuff")

class GitFileDiff(BaseModel):
    filepath: Union[str, os.PathLike]
    diff: str

class AllGitFileDiffs(BaseModel):
    diffs: List[GitFileDiff]

def get_git_diffs(git_dir: Union[str, os.PathLike]) -> AllGitFileDiffs:
    try:
        repo = Repo(git_dir)
    except GitCommandError:
        raise ValueError("Invalid Git repository path")

    diffs = []
    staged_files = [item.a_path for item in repo.index.diff("HEAD")]

    for file in staged_files:
        try:
            diff = repo.git.diff("HEAD", file)
            diffs.append(GitFileDiff(filepath=file, diff=diff))
        except GitCommandError:
            pass 
    
    all_diffs = AllGitFileDiffs(diffs=diffs)
    return all_diffs

In [7]:
diffs = get_git_diffs(root_path / "PycharmProjects" / "LLMStuff")

### Gradio Frontend

In [ ]:
with gr.Blocks() as demo:
    client = OpenAI()

    # Init the file explorer
    file_explorer = gr.FileExplorer(render=False)

    # Define all the components here, render them later. Order matters for render.
    repo_path = get_repo_path_textbox()
    
    # For the max tokens
    max_tokens = get_max_tokens_slider()

    # For the model name
    model_names = get_model_names_dropdown()

    # Another component, for checkboxes
    prompt_options = get_prompt_options()

    # You want to ensure that it's rendering after the button is clicked.
    with gr.Tabs() as tabs:
        with gr.TabItem("File Explorer", id=0):
            file_explorer.render()

            # Time to layout the components, within one row
            with gr.Row(equal_height=True, variant="panel"):
                with gr.Column(variant="panel"):
                    # Render repository path
                    repo_path.render()

                    # Trigger the file explorer
                    choose_dir = gr.Button(
                        value="Choose Directory", variant="secondary", scale=1
                    )
                    choose_dir.click(
                        fn=get_chosen_files,
                        inputs=[repo_path],
                        outputs=[file_explorer],
                    )

                with gr.Column(variant="panel"):
                    with gr.Row(equal_height=True, variant="panel"):
                        prompt_options.render()
                        model_names.render()

                    with gr.Row(equal_height=True, variant="panel"):
                        max_tokens.render()

                        goto_chat_tab = gr.Button(value="Start Chat", render=True)
                        goto_chat_tab.click(
                            fn=select_chat_window,
                            inputs=None,
                            outputs=tabs,
                        )

        with gr.TabItem("Chat Window", id=1):

            def respond(msg, history, file_paths, model_name, max_tokens):
                if len(history) == 0: 
                    convo = init_chat_conversation(file_paths=file_paths, code_only=True, user_question=msg)
                else:
                    convo = convert_history_to_convo(history=history)
                    convo.messages.append(get_message_obj(msg, role='user'))

                # Call the OpenAI API
                response = client.chat.completions.create(
                    messages=convo.model_dump()["messages"],
                    model=model_name,
                    stream=True,
                    max_tokens=max_tokens,
                )

                partial_message = ""
                for chunk in response:
                    if chunk.choices[0].delta.content:
                        partial_message = partial_message + chunk.choices[0].delta.content
                        yield partial_message

                #return response.choices[0].message.content
                
            gr.ChatInterface(
                respond,
                additional_inputs=[file_explorer, model_names, max_tokens],
            )


demo.launch(height=1000)